In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# K Means

In [17]:
!pip install spacy
!pip install emoji --upgrade
!pip install --upgrade nltk

!python -m spacy download en_core_web_lg

import pandas as pd
import spacy
import emoji
import re
import random


# Load spaCy model with disabled components for efficiency
nlp = spacy.load('en_core_web_lg')

# Load the dataset
path = ('/content/drive/MyDrive/Data Youtube/Us_trending_data_2022_2023.csv')
#path = ('/content/drive/MyDrive/Data Youtube/US_trending.csv')
df = pd.read_csv(path)

# Set display options
pd.set_option('display.max_colwidth', None)  # No truncation for title
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 15)  # Set the maximum number of rows to display

# Drop duplicates and unnecessary columns
df.drop_duplicates(subset=['title', 'channelTitle'], inplace=True)
df.drop(['channelId'], axis=1, inplace=True)  # Drop the 'channelId' column

print(df['categoryId'].unique())
df.head(20)

2023-12-20 19:02:38.639981: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 19:02:38.640046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 19:02:38.641916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 19:02:40.244837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
[23 17 28 24 20 10 25 22 27  1 19  2 26 29 15]


,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description
0,The Lawsuit of Shane Dawson,2022-02-15,shane,23,2022-02-16,shane|dawson|conspiracy|theories|journalism|investigative journalism|docuseries|documentary,1071503,77929,0,12215,"MERCH! New Mystery Pig Bags Available Now! https://www.shanedawsonmerch.com/ PODCAST IG @ShaneDawsonPodcast https://www.instagram.com/shanedawsonpodcast/?hl=enMy 2nd Youtube Channel https://www.youtube.com/channel/UCAVojJ1k03GZzjSbdXXunkwMy Instagram @Shanedawson https://www.instagram.com/shanedawson/?hl=enIn this video we have fun around the house, get deep about life’s toughest moments, and check to see if the pizza is still wonky! Enjoy!! Ryland’s Vlogs https://www.youtube.com/channel/UCVe_RrR9AMeY6Z8ixOv5gtwRyland’s Main Channel https://www.youtube.com/user/ryanadams7Ryland’s Podcast https://www.youtube.com/channel/UCH2c0ynrljAA-jW8tq3u3HARyland’s Instagram https://www.instagram.com/rylandadams/?hl=enChris’s Channel https://www.youtube.com/c/ChrisBstationChris’s Instagram https://www.instagram.com/chrisbstation/?hl=enMaxine Friend, The Psychic I Mentioned. She’s amazing!https://maxinefriend.com/https://www.instagram.com/maxinefriend/?hl=enMy Brother Jerid’s Instagram https://www.instagram.com/letsmindtravel/?hl=enJerid’s Channel https://www.youtube.com/c/LetsMindTravel/videosBusiness Contact:United Talent AgencyT: 310.273.6700"
1,When the quiet kid plays his music out loud 😂,2022-02-13,Onevilage,23,2022-02-16,Tiktok|Comedy|Foryou|Onevilage|Shorts|Hilarious|Jokes|Viral|Plot twist|Lol|Funny|Don’t flop|Recommend,2381552,289021,0,3069,🆂🆄🅱🆂🅲🆁🅸🅱🅴▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬FOLLOW THESE TO KEEP UP TO DATE:INSTAGRAM ► (onevilage) https://www.instagram.com/onevilage/DISCORD ► https://discord.io/onevilageSNAPCHAT ► (sahdude221): https://www.snapchat.com/add/sahdude221TWITTER ► (onevilage): https://twitter.com/onevilageTWITCH ► (onevilage): https://www.twitch.tv/onevilage▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬#onevilageDance credit: ashleykeno17 😂❤️e
2,Lizard Cupid 2 #shorts,2022-02-13,TheOdd1sOut,23,2022-02-16,[None],1050293,95261,0,1766,#shortsMatt Watts art ➤ https://www.instagram.com/mattwattsart/Emilee Dummer ➤ https://www.instagram.com/edummerart/ Kelly Jensen ➤ https://www.instagram.com/kelly_anne_art/Claire Anne ➤ https://www.instagram.com/claireanneillustrationAmelia Rosenbalm ➤ https://www.instagram.com/galloame/
3,how youtube is when you start watching a show,2022-02-14,Gus Johnson,23,2022-02-16,gus|gus johnson|gus meme|how youtube is when you start a show|meme|new meme|meme playlist|sketch comedy|short funny video|reddit|reddit trending|unusual memes|meme playlist funny|sven johnson|gus and sven|ian kung,292224,26736,0,2923,"please i just wanna watch the show↓COME FOLLOW ME HERE OR I WILL CRY (HARD)↓NEW MERCH: shop.gusjohnson.coGET LIVE TICKETS HERE: https://www.gusjohnson.co/tour- Twitch: https://www.twitch.tv/gusjohnson- Twitter: https://twitter.com/Gusbuckets- Instagram: https://www.instagram.com/johnsongus- Facebook: https://www.facebook.com/gustoonz- Reddit: https://www.reddit.com/r/gusjohnsonGus Johnson plays all sorts of characters, from Mitchell Robbins to JK Rowling to some guy in God's Country to Imbiamba Jombes to filming videos with his mom to pillow guy to the my pool guy to small town reviews to how they use shotguns in movies to the tough kid growing up to your friend who has a knife to spotify ad guy to the how they use shotgun to the gus johnson shotgun to if i was frozone... I don't know why I am writing this in third person. I am Gus. I wrote this. My brother is Sven Johnson, that guy's alright. Guys, if you put all this dumb stuff in the description, it helps your videos. isn't that wild?Thanks for watching and sharing! Don't stab people. I'll see you later."
4,The Metaverse is So Stupid,2022-02-15,Eddy Burback,23,2022-02-16,[None],434369,43331,0,3398,my other channel - https://www.youtube.com/channel/UCJHM5IJ44__fkOxkD4vxoBwmy tw

In [18]:

#print(df.head())
# Define regex pattern for a broader range of emojis
"""
emoji_pattern = re.compile("(["                      # Start of capture group
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"   # Enclosed characters
                           u"\U0001F100-\U0001F1FF"  # Enclosed Alphanumeric Supplement
                           u"\U0001F200-\U0001F2FF"  # Enclosed Ideographic Supplement
                           "])", flags=re.UNICODE)    # End of capture group
"""
custom_emoji_map = {
    # Add your specific emojis and their text representations here
    "🤓": "nerd_face",
    "☝️": "index_pointing_up",
    "😂": "Face_with_Tears_of_Joy",
    "😭":"Loudly_Crying_Face",
    "💔":"Broken_Heart",
    "😩":"Tired_Face",
    "🔵": "blue_cirlce",
    "🟠":"orange_circle",
    "💨":"zoom_smoke",
    "😱":"Face_Screaming_in_Fear",
    "🙄":"Face_with_Rolling_Eyes",
    "💅":"nails",
    "😍":"smiling_face_with_heart_eyes",
    "✈️":"airplane",
    "🅿️":"parking",
    "💅": "nail_care",
    "🤔": "thinking",
    "🤦": "person_facepalming",
    "🤪": "zany_face",
    "🇨": "regional_indicator_c",
    "🧟": "zombie",
    "🎉": "tada",
    "🧿": "nazar_amulet",
    "🇱": "regional_indicator_l",
    "👆": "point_up_2",
    "🤬": "face_with_symbols_over_mouth",
    "🥛": "milk",
    "🐰": "rabbit",
    "🤯": "exploding_head",
    "👹": "japanese_ogre",
    "🛡️": "shield",
    "🥇": "first_place",
    "🕯️": "candle",
    "🎈": "balloon",
    "🎥": "movie_camera",
    "🦝": "raccoon",
    "💋": "kiss",
    "⭐": "star",
    "👨": "man",
    "🎊": "confetti_ball",
    "😎": "sunglasses",
    "😫": "tired_face",
    "💪": "muscle",
    "🌲": "evergreen_tree",
    "🇿": "regional_indicator_z",
    "💇": "person_getting_haircut",
    "👀": "eyes",
    "😐": "neutral_face",
    "💍": "ring",
    "🥐": "croissant",
    "🗨️": "speech_left",
    "😡": "rage",
    "🏆": "trophy",
    "🏈": "football",
    "🎃": "jack_o_lantern",
    "🇲": "regional_indicator_m",
    "👠": "high_heel",
    "🤘": "metal",
    "🧍": "person_standing",
    "🫒": "olive",
    "🥞": "pancakes",
    "🐭": "mouse",
    "🇭": "regional_indicator_h",
    "🙌": "raised_hands",
    "🥸": "disguised_face",
    "🥘": "shallow_pan_of_food",
    "🎁": "gift",
    "🌊": "ocean",
    "👶": "baby",
    "⚽": "soccer",
    "🧐": "face_with_monocle",
    "💦": "sweat_drops",
    "😵": "dizzy_face",
    "👔": "necktie",
    "⛳": "golf",
    "🥚": "egg",
    "😅": "sweat_smile",
    "🇹": "regional_indicator_t",
    "❌": "x",
    "🛠️": "tools",
    "🧘": "person_in_lotus_position",
    "🌺": "hibiscus",
    "💩": "poop",
    "🎡": "ferris_wheel",
    "🦴": "bone",
    "🇳": "regional_indicator_n",
    "🚛": "articulated_lorry",
    "🤷": "person_shrugging",
    "😀": "grinning",
    "🤖": "robot",
    "🏔️": "mountain_snow",
    "💛": "yellow_heart",
    "💌": "love_letter",
    "🍍": "pineapple",
    "📔": "notebook_with_decorative_cover",
    "🦜": "parrot",
    "🇯": "regional_indicator_j",
    "😬": "grimacing",
    "🤩": "star_struck",
    "🔪": "knife",
    "🗣️": "speaking_head",
    "🌈": "rainbow",
    "🦯": "probing_cane",
    "😿": "crying_cat_face",
    "🪐": "ringed_planet",
    "💔": "broken_heart",
    "👾": "space_invader",
    "🐝": "bee",
    "🌴": "palm_tree",
    "🍕": "pizza",
    "🏌️": "person_golfing",
    "🐷": "pig",
    "🐐": "goat",
    "🤑": "money_mouth",
    "💤": "zzz",
    "💘": "cupid",
    "🔴": "red_circle",
    "📼": "vhs",
    "🐟": "fish",
    "😮": "open_mouth",
    "🚔": "oncoming_police_car",
    "🤣": "rofl",
    "🍭": "lollipop",
    "🍄": "mushroom",
    "😂": "joy",
    "🌅": "sunrise",
    "⚡": "zap",
    "😩": "weary",
    "🌎": "earth_americas",
    "💫": "dizzy",
    "🌌": "milky_way",
    "📰": "newspaper",
    "👁️": "eye",
    "🫖": "teapot",
    "😈": "smiling_imp",
    "🤫": "shushing_face",
    "🦖": "t_rex",
    "🦇": "bat",
    "🍊": "tangerine",
    "🍳": "cooking",
    "🥳": "partying_face",
    "🍖": "meat_on_bone",
    "💕": "two_hearts",
    "🧈": "butter",
    "🎂": "birthday",
    "🍞": "bread",
    "👼": "angel",
    "🌀": "cyclone",
    "🇵": "regional_indicator_p",
    "✈️": "airplane",
    "𝗦": "mathematical_sans_serif_bold_capital_S",
    "𝘥": "mathematical_sans_serif_italic_small_D",
    "𝘳": "mathematical_sans_serif_italic_small_R",
    "𝐙": "mathematical_bold_capital_Z",
    "𝘸": "mathematical_sans_serif_italic_small_W",
    "𝐓": "mathematical_bold_capital_T",
    "𝗧": "'mathematical_sans_serif_bold_capital_T",
    "𝙈": "mathematical_sans_serif_bold_italic_capital_M",
    "𝐔": "mathematical_bold_capital_U",
    "𝗡": "mathematical_sans_serif_bold_capital_N",
    "𝙀": "mathematical_sans_serif_bold_italic_capital_E",
    "𝐎": "mathematical_bold_capital_O",
    "🫶": "heart_hands",
    "♥": "black_heart_suit",
    "❤": "heavy_black_heart",
    "☠": "skull_and_crossbones",
    "🫣": "face_with_peeking_eye",
    "𝗟": "mathematical_sans_serif_bold_capital_L",
    "𝗔": "mathematical_sans_serif_bold_capital_A",
    "𝗖": "mathematical_sans_serif_bold_capital_C",
    "㎡": "square_m_squared",
    "♪": "eighth_note",
    "★": "black_star",
    "🫡": "saluting_face",
    "🥹": "face_holding_back_tears",
    #"ㅣ": "hangul_letter_I",
    "＜": "fullwidth_less_than_sign",
    "～": "fullwidth_tilde",
    "♀": "female_sign",
   "﹥": "small_greater_than_sign",
    "｜": "fullwidth_vertical_line",
    "𝘴": "mathematical_sans_serif_italic_small_S",
    "⛏": "pick",
    "𝘩": "mathematical_sans_serif_italic_small_H",
    "𝘢": "mathematical_sans_serif_italic_small_A",
     "🦲": "emoji_component_bald",
    "👑": "crown",
    "👐": "open_hands",
    "🍦": "icecream",
    "🗿": "moyai",
    "😏": "smirk",
    "🧸": "teddy_bear",
    "😜": "stuck_out_tongue_winking_eye",
    "🇧": "regional_indicator_b",
    "🍿": "popcorn",
    "🔨": "hammer",
    "🎤": "microphone",
    "🇮": "regional_indicator_i",
    "𝐃": "mathematical_bold_capital_D",
    "🚨": "rotating_light",
    "🎀": "ribbon",
    "🗺️": "map",
    "💒": "wedding",
    "✧": "white_four_pointed_star",
    "𝘾": "mathematical_sans_serif_bold_italic_capital_C",
    "🍀": "four_leaf_clover",
    "🔥": "fire",
    "😭": "sob",
    "🇬": "regional_indicator_g",
    "🐶": "dog",
    "🤝": "handshake",
    "🥕": "carrot",
    "💆": "person_getting_massage",
    "🏡": "house_with_garden",
    "😢": "cry",
    "💶": "euro",
    "🚀": "rocket",
    "🇷": "regional_indicator_r",
    "𝐍": "mathematical_bold_capital_N",
    "𝙍": "mathematical_sans_serif_bold_italic_capital_R",
    "►": "black_right_pointing_pointer",
    "🤍": "white_heart",
    "👊": "punch",
    "👸": "princess",
    "⚠": "warning_sign",
    "🍸": "cocktail",
    "😲": "astonished",
    "🎓": "mortar_board",
    "🌋": "volcano",
    "🇩": "regional_indicator_d",
    "💙": "blue_heart",
    "😤": "triumph",
    "🙈": "see_no_evil",
    "🙄": "rolling_eyes",
    "𝗘": "mathematical_sans_serif_bold_capital_E",
    "👏": "clap",
    "🐸": "frog",
    "𝐁": "mathematical_bold_capital_B",
    "😳": "flushed",
    "𝗥": "mathematical_sans_serif_bold_capital_R",
    "🍓": "strawberry",
    "🎯":"dart" ,
    "🍟":"fries",
    "🔮":"crystal_ball",
    "🇸":"regional_indicator_s",
    "💰":"moneybag",
    "☕":"coffee",
    "🥥":"coconut",
    "🏀":"basketball",
    "🤐":"zipper_mouth",
    "💗":"heartpulse",
    "🏳":"flag_white",
    "☝":"Index_Pointing_Up",
    "✿":"soccerball",
    "💡":"bulb",
    "☀":"white_sun",
    "👻":"ghost",
    "🎄":"christmas_tree",
    "🥺":"pleading_face",
    "🏥":"hospital",
    "🔆":"high_brightness",
    "🎲":"game_dice",
    "🚗":"red_car",
    "🍌":"banana",
    "🏠":"house",
    "🚫":"no_entry_sign",
    "😱":"scream",
    "🚂":"steam_locomotive",
    "🏝":"island",
    "🎼":"musical_score",
    "👜":"handbag",
    "🥵":"hot_face",
    "♠":"BLACK_SPADE_SUIT",
    "🤜":"right_facing_fist",
    "𝗠":"MATHEMATICAL_SANS_SERIF_BOLD_CAPITAL_M",
    "🦂":"scorpion",
    "🤺":"person_fencing",
    "🤢":"nauseated_face",
    "💖":"sparkling_heart",
    "👄":"lips",
    "🇴":"regional_indicator_o",
    "🍋":"lemon",
    "𝘵":"MATHEMATICAL_SANS_SERIF_ITALIC_SMALL_T",
    "🌶️":"hot_pepper",
    "💯":"100",
    "🏁":"checkered_flag",
    "🩸":"drop_of_blood",
    "👍":"thumbsup",
    "👮":"police_officer",
    "🦓":"zebra",
    #"へ":"HIRAGANA_LETTER_HE",
    "𝐆":"MATHEMATICAL_BOLD_CAPITAL_G",
    "🤟":"love_you_gesture",
    "🇫":"regional_indicator_f",
    "🐊":"crocodile",
    "🇽":"regional_indicator_x",
    "🤭":"face_with_hand_over_mouth",
    "💸":"money_with_wings",
    "✰":"SHADOWED_WHITE_STAR",
    "😞":"disappointed",
    "🪚":"carpentry_saw",
    "🎵":"musical_note",
    "😰":"cold_sweat",
    "🚘":"oncoming_automobile",
    "🇪":"regional_indicator_e",
    "🟥":"red_square",
    "⚔":"CROSSED_SWORDS",
    "🇺":"regional_indicator_u",
    "🥰":"smiling_face_with_3_hearts",
    "𝐀":"MATHEMATICAL_BOLD_CAPITAL_A",
    "❗":"exclamation",
    "🥴":"woozy_face",
    "𝘼":"MATHEMATICAL_SANS_SERIF_BOLD_ITALIC_CAPITAL_A",
    "🐱":"cat",
    "🎆":"fireworks",
    "１":"FULLWIDTH_DIGIT_ONE",
    "🦋":"butterfly",
    "♡":"WHITE_HEART_SUIT",
    "🏰":"european_castle",
    "🍻":"beers",
    "🌸":"cherry_blossom",
    "𝘪":"MATHEMATICAL_SANS_SERIF_ITALIC_SMALL_I",
    "𝐒":"MATHEMATICAL_BOLD_CAPITAL_S",
    "🥊":"boxing_glove",
    "😁":"grin",
    "🌿":"herb",
    "🤏":"pinching_hand",
    "🌟":"start2",
    "𝗜":"MATHEMATICAL_SANS_SERIF_BOLD_CAPITAL_I",
    #"レ":"KATAKANA_LETTER_RE",
    "⤴":"ARROW_POINTING_RIGHTWARDS_THEN_CURVING_UPWARDS",
    "💀":"skull",
    "♬":"BEAMED_SIXTEENTH_NOTES",
    "🥲":"smiling_face_with_tear",
    "😉":"wink",
    "⛰":"MOUNTAIN",
    "🎮":"video_game",
    "💥":"boom",
    "🇦":"regional_indicator_a",
    "🎨":"art",
    "𝘧":"MATHEMATICAL_SANS_SERIF_ITALIC_SMALL_F",
    "𝘦":"MATHEMATICAL_SANS_SERIF_ITALIC_SMALL_E",
    "🎬":"clapper",
    "🌙":"crescent_moon",
    "🖤":"black_heart",
    "🎙️":"microphone2",
    "😘":"kissing_heart",
    "﹤":"SMALL_LESS_THAN_SIGN",
    #"マ":"KATAKANA_LETTER_MA",
    "🌰":"chestnut",
    "📱":"mobile_phone",
    "🔋":"battery",
    "♨":"HOT_SPRINGS",
    "🧝":"elf",
    "🙏":"pray",
    "🕊️":"dove",
    "📦":"package",
    "𝐄":"MATHEMATICAL_BOLD_CAPITAL_E",
    "🙅":"person_gesturing_no",
    "𝐈":"MATHEMATICAL_BOLD_CAPITAL_I",
    "📸":"camera_with_flash",
    #"ョ":"KATAKANA_LETTER_SMALL_YO"
    "�" : "	REPLACEMENT CHARACTER"
}

In [19]:
# Filter for Shorts
shorts_keywords = ['#Shorts', '#short', 'Shorts', 'shorts']
shorts_pattern = '|'.join(shorts_keywords)
df = df[~df['title'].str.contains(shorts_pattern, na=False) &
        ~df['description'].str.contains(shorts_pattern, na=False) &
        ~df['channelTitle'].str.contains(shorts_pattern, na=False)]

#import emoji

# Function to insert spaces between consecutive emojis
def add_spaces_between_emojis(text):
    spaced_text = ""
    previous_char = None

    for char in text:
        if char in emoji.EMOJI_DATA and previous_char in emoji.EMOJI_DATA:
            spaced_text += " " + char
        else:
            spaced_text += char
        previous_char = char

    return spaced_text

# Function to encode emojis using the emoji library and custom_emoji_map
def encode_emojis(text):
    # First, add spaces between adjacent emojis
    spaced_text = add_spaces_between_emojis(text)

    # Use emoji library to replace emojis with their textual representation
    text_with_emojis_encoded = emoji.demojize(spaced_text, delimiters=(" ", " "))

    # Further replace using custom_emoji_map
    for emoji_char, emoji_text in custom_emoji_map.items():
        text_with_emojis_encoded = text_with_emojis_encoded.replace(emoji_char, f" {emoji_text} ")

    return text_with_emojis_encoded



# process_title excluding emojies
def process_text(text):
    # Apply string manipulations before tokenization
    text = text.replace('|', ' ')
    # Process with spaCy
    doc = nlp(text)

    # Tokenize, lemmatize, remove stop words, punctuation, and non-alphabetic tokens
    processed_text = ' '.join([token.lemma_.lower() for token in doc
                               if not token.is_stop and not token.is_punct and token.is_alpha])
    return processed_text

def process_text_with_emojis(text):
    # Apply string manipulations before tokenization
    text = text.replace('|', ' ')
    text = encode_emojis(text)  # Encode emojis

    # Process with spaCy
    doc = nlp(text)

    # Tokenize, lemmatize, remove stop words, punctuation, but keep emojis
    processed_text = ' '.join([token.lemma_.lower() for token in doc
                               if not token.is_stop and not token.is_punct or token.text.startswith(':') and token.text.endswith(':')])
    return processed_text


# Batch processing function with enhanced processing
def process_text_batch(texts, batch_size=1500):
    return [process_text(text) for text in texts]

def clean_tags(tags_string):
    if pd.isna(tags_string):
        return ""
    return ' '.join([tag.strip() for tag in tags_string.split('|')])

# Apply the batch processing to 'title' and 'cleaned_tags'
df['processed_title'] = process_text_batch(df['title'].astype(str).tolist())
df['cleaned_tags'] = df['tags'].apply(clean_tags).apply(lambda x: process_text(x))
df['processed_title_with_emojis'] = process_text_batch(df['title'].astype(str).tolist())

# Applying tag cleaning and trimming spaces in 'cleaned_tags', 'title', and 'processed_title'
df['cleaned_tags'] = df['tags'].apply(clean_tags).str.strip()
df['title'] = df['title'].str.strip()
df['processed_title'] = df['processed_title'].str.strip()
df['processed_title_with_emojis'] = df['processed_title_with_emojis'].str.strip()

# Display the first 20 rows of the DataFrame
#print("First 20 Rows:")
#print(df.iloc[:250, [df.columns.get_loc(c) for c in ('title', 'processed_title', 'cleaned_tags')]])

print("__________________________________________________________________________________________________\n")

# Print the number of rows in the DataFrame
print("Total number of rows in DataFrame:", df.shape[0])

# Print samples of the DataFrame
random_starts = random.sample(range(len(df) - 15), 3)
for start in random_starts:
    print(f"Sample Records starting from index {start}:")
    print(df.iloc[start:start+15, [df.columns.get_loc(c) for c in ('title', 'processed_title', 'cleaned_tags', 'tags')]])
    print("\n---\n")

# Optionally, save to a file
# df.to_csv('/content/drive/My Drive/processed_data.csv', index=False)

__________________________________________________________________________________________________

Total number of rows in DataFrame: 25227
Sample Records starting from index 1072:
                                                                           title                                      processed_title                                                                                                                                                                                                                                                                                                                                                                                                                                                                cleaned_tags                                                                                                                                                                                                                                 

In [20]:
"""
def find_unencoded_emojis(processed):
    # Find any characters in processed text that are still emojis
    unencoded_emojis = [char for char in processed if char in emoji.EMOJI_DATA]

    return unencoded_emojis

# Apply the function to the DataFrame
df['unencoded_emojis'] = df['processed_title'].apply(find_unencoded_emojis)

# Filter to find rows with unencoded emojis
rows_with_unencoded_emojis = df[df['unencoded_emojis'].apply(lambda x: len(x) > 0)]

# Display the results
print("Titles with Unencoded Emojis:")
print(rows_with_unencoded_emojis[['title', 'processed_title', 'unencoded_emojis']])
print("Total number of rows in DataFrame:", rows_with_unencoded_emojis.shape[0])
"""

'\ndef find_unencoded_emojis(processed):\n    # Find any characters in processed text that are still emojis\n    unencoded_emojis = [char for char in processed if char in emoji.EMOJI_DATA]\n\n    return unencoded_emojis\n\n# Apply the function to the DataFrame\ndf[\'unencoded_emojis\'] = df[\'processed_title\'].apply(find_unencoded_emojis)\n\n# Filter to find rows with unencoded emojis\nrows_with_unencoded_emojis = df[df[\'unencoded_emojis\'].apply(lambda x: len(x) > 0)]\n\n# Display the results\nprint("Titles with Unencoded Emojis:")\nprint(rows_with_unencoded_emojis[[\'title\', \'processed_title\', \'unencoded_emojis\']])\nprint("Total number of rows in DataFrame:", rows_with_unencoded_emojis.shape[0])\n'

In [21]:
from typing import List

# Function for POS Tagging in Batches
def pos_tagging_batch(texts: List[str], batch_size=1500):
    pos_tags = []
    for doc in nlp.pipe(texts, batch_size=batch_size):
        pos_tags.append([(token.text, token.pos_) for token in doc])
    return pos_tags

# Function for Noun Phrase Extraction in Batches
def extract_noun_phrases_batch(texts: List[str], batch_size=1500):
    noun_phrases = []
    for doc in nlp.pipe(texts, batch_size=batch_size):
        noun_phrases.append([chunk.text for chunk in doc.noun_chunks])
    return noun_phrases

# Function for Entity Detection in Batches
def detect_entities_batch(texts: List[str], batch_size=1500):
    entities = []
    for doc in nlp.pipe(texts, batch_size=batch_size):
        entities.append([(entity.text, entity.label_) for entity in doc.ents])
    return entities

# Apply batch processing to 'processed_title'
df['title_pos'] = pos_tagging_batch(df['processed_title'].tolist())
df['title_noun_phrases'] = extract_noun_phrases_batch(df['processed_title'].tolist())
df['title_entities'] = detect_entities_batch(df['processed_title'].tolist())


##Adding the the Category Name


In [25]:
#new df for readability

df_encoded = df.drop(["description", "tags"], axis = 1)
# Load the dataset for cat
path = ('/content/drive/MyDrive/Data Youtube/Youtube_dataCat.csv')
df_cat_info = pd.read_csv(path)

# Merge df_encoded and df_cat_info
new_df = pd.merge(df_encoded, df_cat_info, left_on='categoryId', right_on='CategoryID2', how='inner')
new_df = new_df.drop(["CategoryID2"],axis = 1 )
# Display the head of the files DataFrame
#print("old data \n")
#print(df_encoded['categoryId'].unique())
#print(df_cat_info['CategoryID2'].unique())

#print("new data \n")
#pd.set_option('display.max_rows', 20)  # or any other number you prefer
#print(new_df['categoryId'].isnull().sum())

#print(new_df['categoryId'].unique())
# Now when you print the DataFrame, it will display up to 100 rows
#print(new_df.head(20))

# Count of null or empty values in 'processed_title'
null_or_empty = new_df['processed_title'].isna() | (new_df['processed_title'] == '')

# Keep only rows where 'processed_title' is not null and not empty
new_df = new_df[~null_or_empty]
#new_df = new_df.drop(["description"], axis =1)

# Reset the index if necessary
new_df.reset_index(drop=True, inplace=True)

print("Total number of rows in DataFrame:", new_df.shape[0])

#print(new_df.head(500))
print(new_df.head(25))

#new_df.to_csv('new_df.csv', index=False)

#from google.colab import files
#files.download('new_df.csv')

Total number of rows in DataFrame: 25173
                                                                                                  title publishedAt          channelTitle  categoryId trending_date  view_count   likes  dislikes  comment_count                                                                       processed_title                                                                                                                                                                                                           cleaned_tags                                                           processed_title_with_emojis                                                                                                                                                                                                                                             title_pos                     title_noun_phrases                                                                               

##word embeding & model building

In [27]:
!pip install pyspark

import pandas as pd
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import udf

# Start a Spark session
spark = SparkSession.builder.appName('YouTubeDataProcessing').getOrCreate()

# Convert date columns to Timestamp
new_df['publishedAt'] = pd.to_datetime(new_df['publishedAt'])
new_df['trending_date'] = pd.to_datetime(new_df['trending_date'])

# Function to convert complex structures to string
def list_to_string(lst):
    return str(lst)

list_to_string_udf = udf(list_to_string, types.StringType())

# Apply conversion for complex columns
new_df['title_pos'] = new_df['title_pos'].apply(list_to_string)
new_df['title_noun_phrases'] = new_df['title_noun_phrases'].apply(list_to_string)
new_df['title_entities'] = new_df['title_entities'].apply(list_to_string)

# Drop unnecessary columns
new_df.drop('processed_title_with_emojis', axis=1, inplace=True)

# Rename columns if necessary
new_df.rename(columns={'CategotyNames': 'CategoryNames'}, inplace=True)

# Convert the pandas DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(new_df)

# Convert complex columns using UDFs
spark_df = spark_df.withColumn('title_pos', list_to_string_udf(spark_df['title_pos']))
spark_df = spark_df.withColumn('title_noun_phrases', list_to_string_udf(spark_df['title_noun_phrases']))
spark_df = spark_df.withColumn('title_entities', list_to_string_udf(spark_df['title_entities']))

# Verify the DataFrame
spark_df.printSchema()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=552cd99ff2ba099026c128ede08f988ebdc5083858ebeaa9ba1b428d3aa58121
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
root
 |-- title: string (nullable = true)
 |-- publishedAt: timestamp (nullable = true)
 |-- channelTitle: string (nullable = true)
 |-- categoryId: long (nullable = true)
 |-- trending_date: timestamp (nullable = true)
 |-- view_count: long (nullable = true)
 |-- likes: long (nullable = true)
 |-- dislikes: long (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- processed_title: string (nullable = true)
 |-- cleaned_tags: string (nullable = true)
 |-- title_pos: string (nullable = true)
 |-- title_noun_phrases: string (nullable = true)
 |-- title_entities: stri

In [ ]:
"""import numpy as np
import gc

def get_combined_features(row):
    # Convert 'POS Tags' and 'Entities' lists into strings
    pos_string = ' '.join(['_'.join([t[0], t[1]]) for t in row['title_pos']])
    entities_string = ' '.join([e[0] for e in row['title_entities']]) # Assuming you want just the text part of the entity

    # Join 'Noun Phrases' into a single string
    noun_phrases_string = ' '.join(row['title_noun_phrases'])

    # Generate word embeddings for 'Processed Title' and the other features
    title_vector = nlp(row['processed_title']).vector
    pos_vector = nlp(pos_string).vector
    noun_phrases_vector = nlp(noun_phrases_string).vector
    entities_vector = nlp(entities_string).vector

    # Combine all feature vectors
    combined_vector = np.hstack([title_vector, pos_vector, noun_phrases_vector, entities_vector])
    return combined_vector

def process_in_batches(df, batch_size):
    combined_vectors = []
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        processed_batch = batch.apply(get_combined_features, axis=1)
        combined_vectors.extend(processed_batch)
        gc.collect()
    return np.array(combined_vectors)

def get_tag_embeddings(text):
    return nlp(text).vector

# Load your DataFrame here
# new_df = pd.read_csv('your_dataframe.csv')

# Adjust batch size based on your system's capability
batch_size = 1500

# Apply the function to get vectors for cleaned_tags
new_df['tag_embeddings'] = new_df['cleaned_tags'].apply(get_tag_embeddings)

# Process in batches
combined_vectors = process_in_batches(new_df, batch_size)
"""

In [ ]:
"""
from sklearn.preprocessing import OneHotEncoder
# Add the combined vectors as a new column to your DataFrame
new_df['combined_vector'] = list(combined_vectors)

print(new_df.head(15))
# Initialize and apply OneHotEncoder to categoryNames

encoder = OneHotEncoder(sparse=False)
category_ohe = encoder.fit_transform(new_df[['CategotyNames']])
new_df['category_ohe'] = list(category_ohe)

# Function to combine all features including tag embeddings and category OHE
def combine_all_features(row):
    combined_vector = np.hstack([
        row['combined_vector'],  # Combined vector from previous function
        np.array(row['tag_embeddings']),
        np.array(row['category_ohe'])
    ])
    return combined_vector

# Combine all vectors into a single feature vector for each row
new_df['all_features_vector'] = new_df.apply(combine_all_features, axis=1)

#new_df.to_csv('new_df.csv', index=False)

#from google.colab import files
#files.download('new_df.csv')
"""

                                                      title publishedAt          channelTitle  categoryId trending_date  view_count   likes  dislikes  comment_count                  processed_title                                                                                                                                                                                                                                                                                                                                                                                                                                                                cleaned_tags                                                                        title_pos                 title_noun_phrases               title_entities CategotyNames                                                                                                                                                                                        

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
"""
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

# Convert dates to datetime objects (ensure your date columns are in a consistent format)
new_df['publishedAt'] = pd.to_datetime(new_df['publishedAt'])
new_df['trending_date'] = pd.to_datetime(new_df['trending_date'])

# Calculate the difference in days
new_df['days_to_trend'] = (new_df['trending_date'] - new_df['publishedAt']).dt.days
print(new_df.head(15))
"""

                                                      title publishedAt          channelTitle  categoryId trending_date  view_count     likes  dislikes  comment_count                  processed_title                                                                                                                                                                                                                                                                                                                                                                                                                                                                cleaned_tags                                                                        title_pos                 title_noun_phrases               title_entities CategotyNames                                                                                                                                                                                      

#model using pySpark

In [ ]:
"""
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# Start Spark session
spark = SparkSession.builder.appName('LinearRegression').getOrCreate()

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(new_df)
"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3a0433e8a476f7362ac028b49d1ead9e642760c02419576aa4a35139f1ad810c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


PySparkTypeError: ignored